In [ ]:
"""
"A topology file is always required for loading data into a Universe. A topology file lists atoms, residues, and their connectivity. MDAnalysis accepts the PSF, PDB, CRD, and GRO formats.

A topology file can then be followed by any number of trajectory files. A trajectory file contains a list of coordinates in the order defined in the topology. If no trajectory files are given, then only a structure is loaded. If multiple trajectory files are given, the trajectories are concatenated in the given order. MDAnalysis accepts single frames (e.g. PDB, CRD, GRO) and timeseries data (e.g. DCD, XTC, TRR, XYZ)."
"""

In [1]:
# NOTE: CODE BELOW IS BASED ON THAT INITIALLY GENERATED BY CG4o
import os, sys
import MDAnalysis as mda
import mdtraj as mdj
from src.utils import cif_parsr as cp

def validate_atom_counts(pdb_path, xtc_path):
    """
    Compare the atom counts and report.
    """
    def get_pdb_atom_count(pdb_path):
        u = mda.Universe(pdb_path)
        return len(u.atoms)
    n_pdb = get_pdb_atom_count(pdb_path)

    def get_xtc_atom_count(xtc_path):
        u = mda.Universe(xtc_path)
        return len(u.atoms)
    n_xtc = get_xtc_atom_count(xtc_path)

    print(f"PDB atoms: {n_pdb}")
    print(f"XTC atoms: {n_xtc}")

    if n_pdb != n_xtc:
        print("Atom counts do NOT match! Aborting.")
        sys.exit(1)
    else:
        print("Atom counts match.")
    return True

def validate_filepaths(fpath: str):
    if not os.path.isfile(fpath):
        print(f"Missing pdb/xtc file: {fpath}")
        sys.exit(1)

def load_universe_if_valid(pdbid_chain):
    """
    Check counts and load MDAnalysis Universe.
    """
    rpath_pdbid_chain = (f'../data/ATLAS_downloads/ATLAS/{pdbid_chain}/protein/{pdbid_chain}')
    pdb_path = f'{rpath_pdbid_chain}.pdb'
    validate_filepaths(pdb_path)
    xtc_path = f'{rpath_pdbid_chain}_prod_R2_fit.xtc'
    validate_filepaths(xtc_path)

    validate_atom_counts(pdb_path, xtc_path)

    u = mda.Universe(pdb_path, xtc_path)
    print("Loaded Universe successfully.")
    # Basic info
    print("Number of atoms:", len(u.atoms))
    print("Number of residues:", len(u.residues))
    print("Number of trajectory frames:", len(u.trajectory))
    print("Time step (ps):", u.trajectory.dt)
    return u

def save_snapshots_as_pdbs(pdbid_chain, univ_pdbid_chain):
    rpath_pdbchain_dir = f'../data/ATLAS_parsed/{pdbid_chain}'
    os.makedirs(rpath_pdbchain_dir, exist_ok=True)

    frame_idx = univ_pdbid_chain.trajectory.frame  # current frame index
    print(f'frame_idx={frame_idx}')
    frame_pdb = f'{rpath_pdbchain_dir}/snapshot_frame{frame_idx}.pdb'
    with mda.Writer(frame_pdb) as W:
        W.write(univ_pdbid_chain.atoms)
    print(frame_pdb)

/Users/beanburger/miniconda3/envs/DJ_Intern/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
pdbid_chain = '1cvr_A'
univ_pdbid_chain = load_universe_if_valid(pdbid_chain)
print(f'Number of residues={len(univ_pdbid_chain.residues)}')
print(f'Number of trajectory frames={len(univ_pdbid_chain.trajectory)}')
univ_pdbid_chain.trajectory[0] # Access first frame coordinates

coords = univ_pdbid_chain.atoms.positions
print(f'Coordinates shape={coords.shape}')
print(f'First 5 atom coordinates:\n {coords[:5]}')

for ts in univ_pdbid_chain.trajectory[:3]:  # iterate over first 3 frames (as demo)
    print(f'Frame {ts.frame}, Time {ts.time:.1f} ps')
    com = univ_pdbid_chain.select_atoms('protein').center_of_mass()
    print(f'Protein center of mass={com}')

save_snapshots_as_pdbs(pdbid_chain, univ_pdbid_chain)

PDB atoms: 6662
XTC atoms: 6662
Atom counts match.
Loaded Universe successfully.
Number of atoms: 6662
Number of residues: 435
Number of trajectory frames: 10001
Time step (ps): 10.0
Number of residues=435
Number of trajectory frames=10001
Total frames: 10001
Coordinates shape=(6662, 3)
First 5 atom coordinates:
 [[54.309998 34.61     44.57    ]
 [54.15     33.623997 44.281998]
 [55.275997 34.61     44.958   ]
 [53.659996 34.739998 45.369995]
 [54.23     35.7      43.51    ]]
Frame 0, Time 0.0 ps
Protein center of mass=[38.03369382 48.99416426 37.20638269]
Frame 1, Time 10.0 ps
Protein center of mass=[38.02607955 49.01046558 37.20671854]
Frame 2, Time 20.0 ps
Protein center of mass=[38.02045058 49.02654678 37.2055083 ]
frame_idx=0
../data/ATLAS_parsed/1cvr_A/snapshot_frame0.pdb


/Users/beanburger/miniconda3/envs/DJ_Intern/lib/python3.13/site-packages/MDAnalysis/core/universe.py:1672: UserWarning: there is no reference attributes in this universe to guess types from
  warnings.warn(str(e))
/Users/beanburger/miniconda3/envs/DJ_Intern/lib/python3.13/site-packages/MDAnalysis/core/universe.py:1672: UserWarning: there is no reference attributes (elements, types, or names) in this universe to guess mass from
  warnings.warn(str(e))
/Users/beanburger/miniconda3/envs/DJ_Intern/lib/python3.13/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/Users/beanburger/miniconda3/envs/DJ_Intern/lib/python3.13/site-packages/MDAnalysis/coordinates/PDB.py:1201: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


In [3]:
rpath_pdbchain_dir = f'../data/ATLAS_parsed/{pdbid_chain}'
rpath_snap_pdb = f'{rpath_pdbchain_dir}/snapshot_frame0.pdb'
traj_pdb_chain = mdj.load_pdb(rpath_snap_pdb)
print(type(traj_pdb_chain))
pdf_chain = cp.parse_pdb_snapshot(traj_pdb_chain)
print(pdf_chain.head())

<class 'mdtraj.core.trajectory.Trajectory'>
    resSeq resName  serial      x      y      z
4        1     TYR       5  5.423  3.570  4.351
25       2     THR      26  5.532  3.948  4.362
41       3     PRO      42  5.236  4.193  4.353
53       4     VAL      54  5.244  4.471  4.093
69       5     GLU      70  5.152  4.814  4.257


In [4]:
# Enumerate aamino acids (if required):
import json
relpath_data = os.path.join('..', 'data')
with open(os.path.join(relpath_data, 'enumeration', 'residues.json'), 'r') as json_f:
    residues_enumerated = json.load(json_f)
pdf_chain = pdf_chain.copy()
pdf_chain.loc[:, 'aa_label_num'] = pdf_chain['resName'].map(residues_enumerated).astype('Int64')

In [5]:
pdf_chain.head()
print(pdf_chain.dtypes)

resSeq            int64
resName          object
serial            int64
x               float32
y               float32
z               float32
aa_label_num      Int64
dtype: object


In [6]:
rpath_pdbchain_dir = f'../data/ATLAS_parsed/{pdbid_chain}'
pdf_chain.to_csv(path_or_buf=os.path.join(rpath_pdbchain_dir, f'{pdbid_chain}.ssv'), sep=' ', index=False)

In [8]:
n_frames = len(univ_pdbid_chain.trajectory)
print("Total frames:", n_frames)

Total frames: 10001
